### **CNN to classify vehicles and animals**

In [10]:
import numpy as np 
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [12]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)


In [13]:
image , label = train_data[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [14]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [15]:

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 12, 5)  # Input: (3, 32, 32), Output: (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2)    # Output: (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # Output: (24, 10, 10), after pooling: (24, 5, 5)
        
        # Calculate the input size for the first fully connected layer
        fc1_input_size = 24 * 5 * 5
        
        self.fc1 = nn.Linear(fc1_input_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Output: (12, 14, 14)
        x = self.pool(F.relu(self.conv2(x)))  # Output: (24, 5, 5)
        x = torch.flatten(x, 1)               # Flatten to (batch_size, 24*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
net = NeuralNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
for epoch in range(4):
    print(f'Training Epoch {epoch+1}...')
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch+1}, {i+1}] loss: {running_loss/2000}')
            running_loss = 0.0
print('Finished Training')

Training Epoch 1...
[1, 2000] loss: 2.186082182586193
[1, 4000] loss: 1.8180919324755669
[1, 6000] loss: 1.6353896393328906
[1, 8000] loss: 1.5320133616030216
[1, 10000] loss: 1.4675883934050797
[1, 12000] loss: 1.4094726651608944
Training Epoch 2...
[2, 2000] loss: 1.3158442847169936
[2, 4000] loss: 1.2927929217424243
[2, 6000] loss: 1.271819129049778
[2, 8000] loss: 1.2505593682266771
[2, 10000] loss: 1.2294249670505524
[2, 12000] loss: 1.194690998679027
Training Epoch 3...
[3, 2000] loss: 1.1232161295991392
[3, 4000] loss: 1.0854115597233176
[3, 6000] loss: 1.1043611744428055
[3, 8000] loss: 1.0968646515794098
[3, 10000] loss: 1.0930686998665333
[3, 12000] loss: 1.072778783368878
Training Epoch 4...
[4, 2000] loss: 0.9913643083069473
[4, 4000] loss: 0.9861639241976663
[4, 6000] loss: 1.0296389082334936
[4, 8000] loss: 0.9926068644504994
[4, 10000] loss: 0.9955419842321426
[4, 12000] loss: 0.9866812988868915
Finished Training


In [18]:
torch.save(net.state_dict(), 'trained_net.pth')

In [19]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [20]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 64.7%


In [21]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image).unsqueeze(0)
    return image

image_paths = ['image1.png', 'image2.png']
images = [load_image(img_path) for img_path in image_paths]

images_tensor = torch.cat(images, dim=0)

net.eval()
with torch.no_grad():
    output = net(images_tensor)
    _, predicted = torch.max(output, 1)
    for i, pred in enumerate(predicted):
        print(f'Image {i+1}: Predicted class - {class_names[pred.item()]}')

Image 1: Predicted class - deer
Image 2: Predicted class - plane
